File Preprocessing

In [77]:
!pip install -U numpy
!pip install pyldavis

Requirement already up-to-date: numpy in /opt/conda/lib/python3.7/site-packages (1.16.3)


In [3]:
import os
import codecs
import spacy
import numpy
import gensim
from spacy import displacy
import pandas as pd
import itertools as it

In [4]:
# PICK THE LANGUAGE OF THE CORPUS YOU WISH TO EXAMINE
language = 'de'

In [5]:
%%time
if language == 'fr':
    print('loading french model')
    nlp = spacy.load("fr_core_news_md")
elif language == 'de':
    print ('loading german model')
    nlp = spacy.load("fr_core_news_md")   


loading german model
CPU times: user 25.4 s, sys: 1.62 s, total: 27 s
Wall time: 27 s


We open the preprocessed de file

In [6]:
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

In [9]:
directory = "../corpus_samples/processed/"
filename = directory + language +'.txt'

intermediate_directory = '../corpus_samples/lemmatized/' + language + '/'

def line_page(filename):
    """
    generator function to read in pages from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename, encoding='utf_8') as f:
        for page in f:
            yield page.replace('\\r', '\r')

def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_page in nlp.pipe(line_page(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_page.sents:
            yield u' '.join([token.lemma_ for token in sent])


unigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'unigram_sentences_all.txt')

with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(filename):
        f.write(sentence + '\n')

In [10]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [11]:
for unigram_sentence in it.islice(unigram_sentences, 0, 240):
    print(u' '.join(unigram_sentence))
    print(u'')

auf

Sternwarte Wien ander durch VON Oarr auf Sternwarte München ausgeführt worden war Das durch Konstanten gramme val abs Wien M.-G. i. m cm sek-2 definiert Wiener gramm System ist bisher allgemein al Grundlage für relativen gramm Messungen beibehalten

worden näherungsweise ermittelten

Reduktionsbeträge Beobachtungsreihen auf dasselbe haben

Herrn

Prof helmert dazu gedient eine diesem System angepaßte

Normalformel

abzuleiten seit zur

Darstellung Verlaufs

Schwerkraft an Erdoberfläche verwendet wird

In oben erwähnten Berichte wie

Herr Prof helmert jedoch schon darauf hin daß Wiener Fundamentalkonstante nach seinen auf andere Material gestützten Untersuchungen wahrscheinlich einer Korrektion cm sek-2 bedürfe ihm älteren meist beträchtlichen systematischen Fehlern behafteten absoluten

Messungen zur

Ableitung einer

Normalwertes gramm nicht geeignet erschienen so leitete er im jahre eine absolute Bestimmung gramme für Potsdam Wege

bei dieser über jahre ausgedehnten besonder auf

In [12]:
bigram_model_filepath = os.path.join(intermediate_directory, 'bigram_model')

In [13]:
bigram_model = Phrases(unigram_sentences)

bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

In [14]:
bigram_sentences_filepath = os.path.join(intermediate_directory,
                                         'bigram_sentences_all.txt')
with codecs.open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = u' '.join(bigram_model[unigram_sentence])
        f.write(bigram_sentence + '\n')

In [15]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [16]:
for bigram_sentence in it.islice(bigram_sentences, 0, 240):
    print(u' '.join(bigram_sentence))
    print(u'')

auf

Sternwarte Wien ander durch VON Oarr auf Sternwarte München ausgeführt worden war Das durch Konstanten gramme val abs Wien M.-G. i. m cm sek-2 definiert Wiener gramm System ist bisher allgemein al Grundlage für relativen gramm Messungen beibehalten

worden näherungsweise ermittelten

Reduktionsbeträge Beobachtungsreihen auf dasselbe haben

Herrn

Prof helmert dazu gedient eine diesem System angepaßte

Normalformel

abzuleiten seit zur

Darstellung Verlaufs

Schwerkraft an Erdoberfläche verwendet wird

In oben erwähnten Berichte wie

Herr Prof helmert jedoch schon darauf hin daß Wiener Fundamentalkonstante nach seinen auf andere Material gestützten Untersuchungen wahrscheinlich einer Korrektion cm sek-2 bedürfe ihm älteren meist beträchtlichen systematischen Fehlern behafteten absoluten

Messungen zur

Ableitung einer

Normalwertes gramm nicht geeignet erschienen so leitete er im jahre eine absolute Bestimmung gramme für Potsdam Wege

bei dieser über jahre ausgedehnten besonder auf

In [17]:
trigram_model_filepath = os.path.join(intermediate_directory,
                                      'trigram_model_all')

trigram_model = Phrases(bigram_sentences)

trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

trigram_sentences_filepath = os.path.join(intermediate_directory,
                                          'trigram_sentences_all.txt')


with codecs.open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:
        
    for bigram_sentence in bigram_sentences:
            
        trigram_sentence = u' '.join(trigram_model[bigram_sentence])
            
        f.write(trigram_sentence + '\n')

In [18]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [19]:
trigram_dictionary_filepath = os.path.join(intermediate_directory,
                                           'trigram_dict_all.dict')

In [20]:
#extreme filters hyperparams
no_below = 4
no_above = 0.5

trigram_pages = LineSentence(trigram_sentences_filepath)

# learn the dictionary by iterating over all of the pages
trigram_dictionary = Dictionary(trigram_pages)
    
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
trigram_dictionary.filter_extremes(no_below, no_above)
trigram_dictionary.compactify()

trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

In [21]:
trigram_bow_filepath = os.path.join(intermediate_directory,
                                    'trigram_bow_corpus_all.mm')

In [22]:
def trigram_bow_generator(filepath):
    """
    generator function to read pages from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [23]:
MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_sentences_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

In [24]:
lda_model_filepath = os.path.join(intermediate_directory, 'lda_model_all')

In [25]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
        
    # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
    lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=3,
                           id2word=trigram_dictionary,
                           workers=3)
    
lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

CPU times: user 70 ms, sys: 170 ms, total: 240 ms
Wall time: 360 ms


In [26]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [27]:
explore_topic(topic_number=2)

term                 frequency

Messungen            0.086
dieser               0.069
über                 0.066
eine                 0.063
Kommission           0.050
sich                 0.049
durch                0.040
Internationalen      0.037
worden               0.036
ist                  0.032
Herr                 0.032
schon                0.028
Berlin               0.028
das                  0.027
einer                0.025
ein                  0.024
Permanenten          0.024
auf                  0.023
gramm                0.023
al                   0.019
Prof                 0.019
docteur              0.018
Polhöhe              0.017
Herrn                0.016
jahre                0.015


In [28]:
LDAvis_data_filepath = os.path.join(intermediate_directory, 'ldavis_prepared')

In [29]:
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,
                                              trigram_dictionary)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [30]:
pyLDAvis.display(LDAvis_prepared)